In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, CuDNNLSTM, Multiply,Dropout
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
from keras.callbacks import TensorBoard
import numpy as np
import pickle
import time
import tensorflow as tf

C:\Users\djaym7\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def loadData(id):
    with open(f'C:\\Users\\djaym7\\Desktop\\Github\\EmotionRecognition\\data 02\\y_train_{id}','rb') as f:
        y_train=np.array(pickle.load(f))
    with open(f'C:\\Users\\djaym7\\Desktop\\Github\\EmotionRecognition\\data 02\\y_test_{id}','rb') as f:
        y_test=np.array(pickle.load(f))
    with open(f'C:\\Users\\djaym7\\Desktop\\Github\\EmotionRecognition\\data 02\\X_train_{id}','rb') as f:
        X_train = np.array(pickle.load(f))
    with open(f'C:\\Users\\djaym7\\Desktop\\Github\\EmotionRecognition\\data 02\\X_test_{id}','rb') as f:
        X_test = np.array(pickle.load(f))

    

    from sklearn.preprocessing import LabelBinarizer
    encoder = LabelBinarizer()
    y_train = encoder.fit_transform(y_train)
    y_test = encoder.fit_transform(y_test)


    print('X_train:',X_train.shape)
    print('X_test:',X_test.shape)
    print('y_train',y_train.shape) 
    print('y_test',y_test.shape) 

   
    return X_train,y_train,X_test,y_test


In [3]:

total_num_people=[]
for i in range(1,92):
    if i <10:
        total_num_people.append('100'+str(i))
    else:
        total_num_people.append('10'+str(i))

for i in total_num_people:
    if int(i)==1011:
        X_train=[]
        X_test=[]
        y_train=[]
        y_test=[]
        X_train,y_train,X_test,y_test=loadData(i)
    else:
        continue


    #load data here
    #input_shape = batch_size,time_steps,input_dim

    #batch_size = X_train.shape[0]   #automatically taken no need to specify
    time_steps = X_train.shape[1]
    input_dim = X_train.shape[2]



    hid_pre_acts = [16]
    hid_post_acts = [96]
    #dense1s = [16,32,64,96,128,256]
    #dense2s = [16,32,64,96,128,256]
    for hid_pre_act in hid_pre_acts:
        for hid_post_act in hid_post_acts:
            # Defined shared layers as global variables
            repeator = RepeatVector(time_steps) #500
            concatenator = Concatenate(axis=-1)
            densor = Dense(1, activation = "relu")
            activator = Activation('softmax', name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
            dotor = Dot(axes = 1)

            #one step attention

            def one_step_attention(a,s_prev):

                s_prev = repeator(s_prev)
                concat = concatenator([a,s_prev])
                e = densor(concat)
                alphas = activator(e)
                context = dotor([alphas, a])

                return context
            #for dense1 in dense1s:
                #for dense2 in dense2s:
            Name = f'FINAL-[16,96] id -{i}---{int(time.time())}'
            tensorboard = TensorBoard(log_dir=f'C:\\Users\\djaym7\\Desktop\\Github\\EmotionRecognition\\logs\\{Name}')
            print(Name)
            post_activation_lstm_cell = CuDNNLSTM(hid_post_act,return_state=True)
            #output_layer = Dense(dense1,activation='relu')
            #onemore = Dense(dense2,activation='relu')
            fin_out = Dense(6,activation='softmax')

                # model

            def model(time_steps,input_dim,hid_post_act,hid_pre_act):

                X = Input(shape =(time_steps,input_dim))
                s0 = Input(shape=(hid_post_act,),name='s0')
                c0 = Input(shape=(hid_post_act,),name='c0')
                s=s0
                c=c0
                outputs=[]



                a = CuDNNLSTM(hid_pre_act,return_sequences=True)(X)
                a = Dropout(0.4)(a)
                #6 is final output size
                for t in range(1):
                    context =one_step_attention(a,s)
                    s, _, c = post_activation_lstm_cell(context, initial_state=[s, c])
                    #out = output_layer(s)
                    #o = onemore(out)
                    k=fin_out(s)
                    outputs.append(k)


                model = Model([X, s0, c0], outputs)

                return model       

            model = model(time_steps,input_dim,hid_post_act,hid_pre_act)

            opt = Adam()
            #opt = Adam(lr = 0.005, beta_1=0.9, beta_2=0.999, decay = 0.01)
            model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

            s0 = np.zeros((X_train.shape[0], hid_post_act))
            c0 = np.zeros((X_train.shape[0], hid_post_act))
            s0_t = np.zeros((X_test.shape[0], hid_post_act))
            c0_t = np.zeros((X_test.shape[0], hid_post_act))
            print(f'id : {i}')
            model.fit([X_train, s0, c0], y_train, epochs=150,validation_data=([X_test,s0_t,c0_t],y_test),callbacks=[tensorboard])
            model.save(f'{i}.h5')


X_train: (7360, 500, 13)
X_test: (82, 500, 13)
y_train (7360, 6)
y_test (82, 6)
FINAL-[16,96] id -1011---1544420670
id : 1011
Train on 7360 samples, validate on 82 samples
Epoch 1/150
7360/7360 [==============================] - 17s 2ms/step - loss: 1.6955 - acc: 0.2870 - val_loss: 1.5407 - val_acc: 0.3659
Epoch 2/150
7360/7360 [==============================] - 15s 2ms/step - loss: 1.5443 - acc: 0.3694 - val_loss: 1.4162 - val_acc: 0.3902
Epoch 3/150
7360/7360 [==============================] - 15s 2ms/step - loss: 1.4931 - acc: 0.3886 - val_loss: 1.3748 - val_acc: 0.4390
Epoch 4/150
7360/7360 [==============================] - 15s 2ms/step - loss: 1.4651 - acc: 0.4043 - val_loss: 1.3725 - val_acc: 0.4878
Epoch 5/150
7360/7360 [==============================] - 15s 2ms/step - loss: 1.4409 - acc: 0.4149 - val_loss: 1.3395 - val_acc: 0.4512
Epoch 6/150
7360/7360 [==============================] - 15s 2ms/step - loss: 1.4305 - acc: 0.4242 - val_loss: 1.3380 - val_acc: 0.4756
Epoch 7/150


7360/7360 [==============================] - 15s 2ms/step - loss: 1.1320 - acc: 0.5526 - val_loss: 1.2198 - val_acc: 0.5244
Epoch 60/150
7360/7360 [==============================] - 15s 2ms/step - loss: 1.1207 - acc: 0.5671 - val_loss: 1.1999 - val_acc: 0.5732
Epoch 61/150
7360/7360 [==============================] - 15s 2ms/step - loss: 1.1294 - acc: 0.5534 - val_loss: 1.3500 - val_acc: 0.4756
Epoch 62/150
7360/7360 [==============================] - 15s 2ms/step - loss: 1.1208 - acc: 0.5693 - val_loss: 1.3183 - val_acc: 0.4878
Epoch 63/150
7360/7360 [==============================] - 15s 2ms/step - loss: 1.1122 - acc: 0.5721 - val_loss: 1.2480 - val_acc: 0.5000
Epoch 64/150
7360/7360 [==============================] - 15s 2ms/step - loss: 1.1107 - acc: 0.5666 - val_loss: 1.3303 - val_acc: 0.4756
Epoch 65/150
7360/7360 [==============================] - 15s 2ms/step - loss: 1.1037 - acc: 0.5715 - val_loss: 1.2725 - val_acc: 0.4878
Epoch 66/150
7360/7360 [==============================

7360/7360 [==============================] - 15s 2ms/step - loss: 1.0285 - acc: 0.5989 - val_loss: 1.2905 - val_acc: 0.5122
Epoch 119/150
7360/7360 [==============================] - 15s 2ms/step - loss: 1.0229 - acc: 0.6098 - val_loss: 1.2787 - val_acc: 0.4756
Epoch 120/150
7360/7360 [==============================] - 15s 2ms/step - loss: 1.0288 - acc: 0.5992 - val_loss: 1.3344 - val_acc: 0.4756
Epoch 121/150
7360/7360 [==============================] - 15s 2ms/step - loss: 1.0253 - acc: 0.6058 - val_loss: 1.2747 - val_acc: 0.4634
Epoch 122/150
7360/7360 [==============================] - 15s 2ms/step - loss: 1.0203 - acc: 0.6060 - val_loss: 1.3654 - val_acc: 0.4878
Epoch 123/150
7360/7360 [==============================] - 15s 2ms/step - loss: 1.0139 - acc: 0.6061 - val_loss: 1.2170 - val_acc: 0.5122
Epoch 124/150
7360/7360 [==============================] - 15s 2ms/step - loss: 1.0084 - acc: 0.6122 - val_loss: 1.2299 - val_acc: 0.5000
Epoch 125/150
7360/7360 [=======================

C:\Users\djaym7\Anaconda3\lib\site-packages\keras\engine\network.py:872: UserWarning: Layer cu_dnnlstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 's0:0' shape=(?, 96) dtype=float32>, <tf.Tensor 'c0:0' shape=(?, 96) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
